In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
from torch.autograd.functional import hessian, jacobian

In [2]:
dtype = torch.float
#device = torch.device("cuda:0")
device = torch.device("cpu")

# Number of points to train on
num_points = 100

# normalized time domain for ODE
t = torch.linspace(0, 1, num_points, device=device, dtype=dtype, requires_grad=True).reshape(-1,1)

# Make a random symetric + real matrix
Q = torch.randn(6,6, device=device, dtype=dtype, requires_grad=False)
A = torch.transpose(Q, 0, 1) + Q 
A.requires_grad = True

# Initial guess for the eigenvalue
v0 = torch.randn(A.size(0), device=device, dtype=dtype, requires_grad=False).reshape(-1,1)

# Trial solution to ODE
def f_trial(t, N):
    return v0 * torch.exp(-t*100) + (1 - torch.exp(-t*100)) * N

# Shape of network
input_dim = 1
hidden_neurons = 100
output_dim = v0.size(0)

# Create random Tensors for weights.
w1 = torch.randn(input_dim, hidden_neurons, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(hidden_neurons, output_dim, device=device, dtype=dtype, requires_grad=True)

In [3]:
#v0.transpose(0,1).mm(A).mm(v0)
- v0 + (v0.transpose(0,1).mm(v0) * A + (1 - v0.transpose(0,1).mm(A).mm(v0) * torch.eye(v0.size(0)))).mm(v0)

tensor([[-14.3586],
        [  7.5750],
        [-12.3006],
        [ 24.4805],
        [-14.4461],
        [-14.7098]], grad_fn=<AddBackward0>)

propose a trial solution to the ODE in the form
$$ 
    f_t(t, P) = v_0 \, e^{-t} + (1-e^{-t}) N(t,P)
$$
Where $N$ denotes the Neural Network, and $P$ its free parameters.

In [4]:
def costfunction(t, N, A, trial_func):
    loss = 0
    
    for i in range(t.size(0)):
        # Compute trial eigenvector & transpose
        v = trial_func(t[i], N[i])
        v_T = v.transpose(0,1)
       
        # Time derivative of trial function
        dfdt, dfdN = jacobian(trial_func, (t[i], N[i]), create_graph=True)  
        
        # right hand side of ODE
        RHS = (- v + (v_T.mm(v) * A  + (1 - v_T.mm(A).mm(v) * torch.eye(v.size(0)))).mm(v))
        
        # Increment the loss function
        loss = loss + (dfdt - RHS).pow(2).mean()
    
    return loss / t.size(0)

In [5]:
learning_rate = 0.00002
for epoch in range(10):
    N_output = F.relu(t.mm(w1)).mm(w2)
    
    # Compute and print loss    
    loss = costfunction(t, N_output, A, f_trial)
    print("loss = ",loss)
    
    # Use autograd to compute the backward pass.
    loss.backward()

    # Update weights using gradient descent
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
    
        # Manually zero the gradients after updating weights
        w1.grad.zero_()
        w2.grad.zero_()

loss =  tensor(7619651., grad_fn=<DivBackward0>)
loss =  tensor(1.4248e+27, grad_fn=<DivBackward0>)
loss =  tensor(109.5872, grad_fn=<DivBackward0>)
loss =  tensor(109.5872, grad_fn=<DivBackward0>)
loss =  tensor(109.5872, grad_fn=<DivBackward0>)
loss =  tensor(109.5872, grad_fn=<DivBackward0>)
loss =  tensor(109.5872, grad_fn=<DivBackward0>)
loss =  tensor(109.5872, grad_fn=<DivBackward0>)
loss =  tensor(109.5872, grad_fn=<DivBackward0>)
loss =  tensor(109.5872, grad_fn=<DivBackward0>)


In [6]:
v_pred = f_trial(t[-1], N_output[-1])
print("before norm: ",v_pred)
v_pred = v_pred/v_pred.norm()
print("after norm", v_pred)

eigal, eigvec = np.linalg.eigh(A.detach().numpy())
print(eigvec)

before norm:  tensor([[ 3.7835e-44,  3.7835e-44,  3.7835e-44,  3.7835e-44,  3.7835e-44,
          3.7835e-44],
        [ 4.2039e-44,  4.2039e-44,  4.2039e-44,  4.2039e-44,  4.2039e-44,
          4.2039e-44],
        [-5.7453e-44, -5.7453e-44, -5.7453e-44, -5.7453e-44, -5.7453e-44,
         -5.7453e-44],
        [-2.3822e-44, -2.3822e-44, -2.3822e-44, -2.3822e-44, -2.3822e-44,
         -2.3822e-44],
        [-1.5414e-44, -1.5414e-44, -1.5414e-44, -1.5414e-44, -1.5414e-44,
         -1.5414e-44],
        [ 2.5223e-44,  2.5223e-44,  2.5223e-44,  2.5223e-44,  2.5223e-44,
          2.5223e-44]], grad_fn=<AddBackward0>)
after norm tensor([[inf, inf, inf, inf, inf, inf],
        [inf, inf, inf, inf, inf, inf],
        [-inf, -inf, -inf, -inf, -inf, -inf],
        [-inf, -inf, -inf, -inf, -inf, -inf],
        [-inf, -inf, -inf, -inf, -inf, -inf],
        [inf, inf, inf, inf, inf, inf]], grad_fn=<DivBackward0>)
[[ 0.2796801   0.34810558  0.05551316 -0.7366981   0.20594901 -0.46084788]
 [ 0.39653